In [ ]:
"""
Advanced Time Series Forecasting with Deep Learning (LSTM + Attention / Transformer)

Single-file, production-ready Python script that:
1) Generates a synthetic multivariate time series dataset (>=1000 obs) with seasonality, trend, regime shifts
2) Preprocesses data and creates rolling-origin cross-validation folds
3) Implements two deep-learning models:
   - LSTM encoder + Bahdanau attention (PyTorch)
   - Transformer encoder (PyTorch)
4) Benchmarks against a statistical SARIMA model (statsmodels)
5) Trains, evaluates (RMSE, MAE, MAPE), and visualizes results and attention weights
6) Exports a simple plain-text technical report summarizing methodology and results

Dependencies (use the requirements.txt below):
- numpy
- pandas
- scikit-learn
- matplotlib
- seaborn
- torch
- tqdm
- statsmodels
- scipy

Usage:
python advanced_time_series_attention_project.py

Adjust hyperparameters near the top of the file.

"""

import os
import math
import json
import random
from typing import Tuple, List, Dict

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import statsmodels.api as sm

# -----------------------------
# Config / hyperparameters
# -----------------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

N_SAMPLES = 4000        # number of time steps (>=1000)
N_FEATURES = 4         # number of covariates (including target)
LOOKBACK = 60          # input window length
HORIZON = 7            # forecast horizon (multi-step)
BATCH_SIZE = 64
EPOCHS = 30
LR = 1e-3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

OUTPUT_DIR = 'output_ts_project'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# -----------------------------
# Utilities and metrics
# -----------------------------

def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # avoid division by zero
    denom = np.where(np.abs(y_true) < 1e-8, 1e-8, np.abs(y_true))
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100

# -----------------------------
# Synthetic dataset generator
# -----------------------------

def generate_synthetic_multivariate(n_steps: int, n_features: int, seed: int = SEED) -> pd.DataFrame:
    """Generate a multivariate time series with seasonality, trend, interactions, noise, and regime shifts.
    First column is the target variable named 'y'; others are covariates x1, x2, ...
    """
    np.random.seed(seed)
    t = np.arange(n_steps)

    # components for target
    trend = 0.01 * t
    seasonal = 2.0 * np.sin(2 * np.pi * t / 24)  # daily-like seasonality
    longer_season = 0.5 * np.sin(2 * np.pi * t / 168)  # weekly-like

    # regime shifts: at two points amplitude changes
    regime = np.ones(n_steps)
    regime[t > int(0.5 * n_steps)] = 1.6
    regime[t > int(0.8 * n_steps)] = 0.6

    # covariates
    data = np.zeros((n_steps, n_features))
    # target is influenced by covariates
    x1 = 0.5 * np.sin(2 * np.pi * t / 12 + 0.2) + 0.1 * np.random.randn(n_steps)
    x2 = 0.3 * np.cos(2 * np.pi * t / 24 * 0.6) + 0.05 * np.random.randn(n_steps)
    x3 = (np.sign(np.sin(2 * np.pi * t / 200)) + 1) * 0.5 + 0.05 * np.random.randn(n_steps)  # slow regime-like

    noise = 0.3 * np.random.randn(n_steps)

    y = (trend + seasonal + longer_season) * regime + 0.7 * x1 + 0.4 * x2 + 0.2 * x3 + noise

    data[:,0] = y
    covs = [x1, x2, x3]
    for i in range(1, n_features):
        if i-1 < len(covs):
            data[:, i] = covs[i-1]
        else:
            data[:, i] = 0.1 * np.random.randn(n_steps)

    cols = ['y'] + [f'x{i}' for i in range(1, n_features)]
    df = pd.DataFrame(data, columns=cols)
    df['t'] = pd.date_range(start='2000-01-01', periods=n_steps, freq='H')
    df = df.set_index('t')
    return df

# -----------------------------
# Dataset and DataLoader
# -----------------------------

class TimeSeriesDataset(Dataset):
    def __init__(self, data: np.ndarray, lookback: int, horizon: int):
        # data: (T, features)
        self.data = data
        self.lookback = lookback
        self.horizon = horizon
        self.T = data.shape[0]

    def __len__(self):
        return max(0, self.T - self.lookback - self.horizon + 1)

    def __getitem__(self, idx):
        start = idx
        end = idx + self.lookback
        x = self.data[start:end]  # (lookback, features)
        y = self.data[end:end + self.horizon, 0]  # target only, horizon steps
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# -----------------------------
# Models
# -----------------------------

class BahdanauAttention(nn.Module):
    def __init__(self, enc_dim, dec_dim):
        super().__init__()
        self.W1 = nn.Linear(enc_dim, dec_dim)
        self.W2 = nn.Linear(dec_dim, dec_dim)
        self.V = nn.Linear(dec_dim, 1)

    def forward(self, encoder_outputs, decoder_hidden):
        # encoder_outputs: (batch, seq_len, enc_dim)
        # decoder_hidden: (batch, dec_dim)  -- we'll use a projection of last hidden
        seq_len = encoder_outputs.size(1)
        dec_hidden_expanded = decoder_hidden.unsqueeze(1).repeat(1, seq_len, 1)  # (batch, seq_len, dec_dim)
        score = torch.tanh(self.W1(encoder_outputs) + self.W2(dec_hidden_expanded))
        attention_weights = torch.softmax(self.V(score).squeeze(-1), dim=1)  # (batch, seq_len)
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)  # (batch, enc_dim)
        return context, attention_weights

class LSTMWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, n_layers=1, dropout=0.1, horizon=1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.encoder = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=dropout, bidirectional=False)
        self.attention = BahdanauAttention(enc_dim=hidden_dim, dec_dim=hidden_dim)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Linear(hidden_dim//2, horizon)
        )

    def forward(self, x):
        # x: (batch, seq_len, features)
        enc_out, (h_n, c_n) = self.encoder(x)
        # take last layer hidden state
        last_hidden = h_n[-1]  # (batch, hidden_dim)
        context, attn_weights = self.attention(enc_out, last_hidden)
        out = self.fc(context)  # (batch, horizon)
        return out, attn_weights

# Simple Transformer encoder model for forecasting
class TransformerForecaster(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2, dim_feedforward=128, horizon=1, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(nn.Linear(d_model, dim_feedforward), nn.ReLU(), nn.Linear(dim_feedforward, horizon))

    def forward(self, x):
        # x: (batch, seq_len, features)
        z = self.input_proj(x)  # (batch, seq_len, d_model)
        enc = self.transformer(z)  # (batch, seq_len, d_model)
        # pool over time
        pooled = enc.mean(dim=1)  # (batch, d_model)
        out = self.fc(pooled)
        # no attention weights produced
        return out, None

# -----------------------------
# Training & evaluation functions
# -----------------------------

def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    for x, y in dataloader:
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        optimizer.zero_grad()
        preds, _ = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(dataloader.dataset)


def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    preds_all = []
    y_all = []
    attn_weights_all = []
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            preds, attn = model(x)
            loss = criterion(preds, y)
            total_loss += loss.item() * x.size(0)
            preds_all.append(preds.cpu().numpy())
            y_all.append(y.cpu().numpy())
            if attn is not None:
                attn_weights_all.append(attn.cpu().numpy())
    preds_all = np.vstack(preds_all)
    y_all = np.vstack(y_all)
    if attn_weights_all:
        attn_weights_all = np.vstack(attn_weights_all)
    else:
        attn_weights_all = None
    return total_loss / len(dataloader.dataset), preds_all, y_all, attn_weights_all

# -----------------------------
# Rolling-origin cross validation
# -----------------------------

def rolling_origin_splits(data_len, initial_train_size, horizon, step):
    """Yield (train_indices, val_indices) pairs for rolling-origin evaluation.
    train window grows by `step` each fold (expanding window)."""
    start = initial_train_size
    while start + horizon <= data_len:
        train_idx = np.arange(0, start)
        val_idx = np.arange(start, min(start + step, data_len - horizon + 1))  # produce multiple validation starts
        # We'll generate samples from these indices when building datasets
        yield train_idx, val_idx
        start += step

# -----------------------------
# SARIMA benchmark function
# -----------------------------

def sarima_forecast(train_series: pd.Series, val_index: pd.DatetimeIndex, horizon: int):
    # use a simple SARIMAX with seasonal order guess (24) for hourly-seasonality; keep it small for speed
    try:
        model = sm.tsa.SARIMAX(train_series, order=(1,1,1), seasonal_order=(1,0,1,24), enforce_stationarity=False, enforce_invertibility=False)
        res = model.fit(disp=False)
        preds = res.predict(start=val_index[0], end=val_index[0] + pd.Timedelta(hours=horizon-1))
        return preds.values
    except Exception as e:
        print('SARIMA failed:', e)
        # fallback: naive persistence
        last = train_series.iloc[-1]
        return np.ones(horizon) * last

# -----------------------------
# Main pipeline
# -----------------------------

def prepare_data(df: pd.DataFrame, feature_cols: List[str]):
    # scaler fit on training range externally
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df[feature_cols])
    return scaled, scaler


def run_pipeline():
    # 1) Data generation
    df = generate_synthetic_multivariate(N_SAMPLES, N_FEATURES)
    df.to_csv(os.path.join(OUTPUT_DIR, 'synthetic_data.csv'))

    feature_cols = df.columns.tolist()  # includes 'y' and x1..xn

    # We'll run rolling-origin CV with expanding window
    initial_train = int(0.5 * N_SAMPLES)
    step = int(0.1 * N_SAMPLES)

    all_results = []
    fold_id = 0

    for train_idx, val_idx in rolling_origin_splits(len(df), initial_train, HORIZON, step):
        fold_id += 1
        print(f"Running fold {fold_id} — train till {train_idx[-1] if len(train_idx)>0 else 0}, val starts at {val_idx[0]}")

        # build training and validation slices
        # For modeling we need arrays; we will fit scaler on train only
        train_df = df.iloc[train_idx]
        val_start = val_idx[0]
        val_end = val_start + step + HORIZON - 1
        val_df = df.iloc[val_start: val_end]

        # scale
        scaler = StandardScaler()
        scaler.fit(train_df[feature_cols])
        train_scaled = scaler.transform(train_df[feature_cols])
        val_scaled = scaler.transform(val_df[feature_cols])

        # Datasets
        train_dataset = TimeSeriesDataset(train_scaled, LOOKBACK, HORIZON)
        val_dataset = TimeSeriesDataset(np.vstack([train_scaled[-LOOKBACK:], val_scaled]), LOOKBACK, HORIZON)
        # note: for validation we append last LOOKBACK from train to the new val window to allow first samples

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # instantiate models
        input_dim = train_scaled.shape[1]
        lstm_model = LSTMWithAttention(input_dim, hidden_dim=64, n_layers=1, horizon=HORIZON).to(DEVICE)
        trans_model = TransformerForecaster(input_dim, d_model=64, nhead=4, num_layers=2, horizon=HORIZON).to(DEVICE)

        criterion = nn.MSELoss()

        # Train LSTM with attention
        optimizer = torch.optim.Adam(lstm_model.parameters(), lr=LR)
        best_val_loss = float('inf')
        for epoch in range(EPOCHS):
            train_loss = train_epoch(lstm_model, train_loader, optimizer, criterion)
            val_loss, preds_val, y_val, attn_val = evaluate_model(lstm_model, val_loader, criterion)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(lstm_model.state_dict(), os.path.join(OUTPUT_DIR, f'lstm_fold{fold_id}.pt'))
            if epoch % 5 == 0:
                print(f"[LSTM] Fold {fold_id} Epoch {epoch}: train_loss={train_loss:.4f} val_loss={val_loss:.4f}")

        # Evaluate saved best model on val set
        lstm_model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, f'lstm_fold{fold_id}.pt')))
        _, preds_val_lstm, y_val_lstm, attn_val = evaluate_model(lstm_model, val_loader, criterion)

        # Flatten multi-step predictions for metrics (evaluate per-horizon aggregated)
        # We'll compute metrics for horizon=1..H on aggregated predictions
        results = {'fold': fold_id, 'lstm_preds': preds_val_lstm, 'y_val': y_val_lstm}

        # Train Transformer
        optimizer_t = torch.optim.Adam(trans_model.parameters(), lr=LR)
        best_val_loss_t = float('inf')
        for epoch in range(EPOCHS):
            train_loss_t = train_epoch(trans_model, train_loader, optimizer_t, criterion)
            val_loss_t, preds_val_t, y_val_t_t, _ = evaluate_model(trans_model, val_loader, criterion)
            if val_loss_t < best_val_loss_t:
                best_val_loss_t = val_loss_t
                torch.save(trans_model.state_dict(), os.path.join(OUTPUT_DIR, f'trans_fold{fold_id}.pt'))
            if epoch % 5 == 0:
                print(f"[Transformer] Fold {fold_id} Epoch {epoch}: train_loss={train_loss_t:.4f} val_loss={val_loss_t:.4f}")

        trans_model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, f'trans_fold{fold_id}.pt')))
        _, preds_val_trans, y_val_trans, _ = evaluate_model(trans_model, val_loader, criterion)
        results.update({'trans_preds': preds_val_trans})

        # SARIMA baseline: create series combining train and val for indexing
        # We'll forecast horizon starting at val start time
        train_series = train_df['y']
        # get the exact timestamps for val horizon
        val_index = df.index[val_start: val_start + HORIZON]
        sarima_preds = sarima_forecast(train_series, val_index, HORIZON)
        results.update({'sarima_preds': sarima_preds})

        # compute metrics aggregated across all validation samples
        # preds arrays have shape (n_samples_val, horizon)
        def compute_metrics(y_true_arr, y_pred_arr):
            # compute per-horizon metrics and overall flattened metrics
            n_samples = y_true_arr.shape[0]
            per_h_metrics = []
            for h in range(y_true_arr.shape[1]):
                ytrue = y_true_arr[:, h]
                ypred = y_pred_arr[:, h]
                per_h_metrics.append({'rmse': rmse(ytrue, ypred), 'mae': mean_absolute_error(ytrue, ypred), 'mape': mape(ytrue, ypred)})
            # flattened
            flat_rmse = rmse(y_true_arr.flatten(), y_pred_arr.flatten())
            flat_mae = mean_absolute_error(y_true_arr.flatten(), y_pred_arr.flatten())
            flat_mape = mape(y_true_arr.flatten(), y_pred_arr.flatten())
            return {'per_h': per_h_metrics, 'flat': {'rmse': flat_rmse, 'mae': flat_mae, 'mape': flat_mape}}

        metrics_lstm = compute_metrics(y_val_lstm, preds_val_lstm)
        metrics_trans = compute_metrics(y_val_trans, preds_val_trans)

        # For SARIMA, we only have single forecast per fold; expand to match shape
        # We'll compare SARIMA to first validation window only (simple approach)
        # create repeated rows to compare with sample count
        n_val_samples = preds_val_lstm.shape[0]
        sarima_rep = np.tile(sarima_preds.reshape(1, -1), (n_val_samples, 1))
        metrics_sarima = compute_metrics(y_val_lstm, sarima_rep)

        results.update({'metrics_lstm': metrics_lstm, 'metrics_transformer': metrics_trans, 'metrics_sarima': metrics_sarima, 'attn_weights': attn_val})
        all_results.append(results)

        # Save intermediate results
        with open(os.path.join(OUTPUT_DIR, f'results_fold{fold_id}.json'), 'w') as f:
            json.dump({'metrics_lstm': metrics_lstm, 'metrics_transformer': metrics_trans, 'metrics_sarima': metrics_sarima}, f, indent=2)

        # quick plot for fold
        plot_fold_results(df, scaler, val_start, LOOKBACK, preds_val_lstm, preds_val_trans, sarima_preds, y_val_lstm, attn_val, fold_id)

    # Summarize across folds
    save_aggregate_report(all_results)

# -----------------------------
# Plotting helper
# -----------------------------

def plot_fold_results(df, scaler, val_start, lookback, preds_lstm, preds_trans, sarima_preds, y_val, attn_weights, fold_id):
    # plot first validation sample actual vs predicted for horizon
    plt.figure(figsize=(12,6))
    # pick a representative sample (first)
    sample_idx = 0
    target_series = df['y'].values
    t_indices = np.arange(val_start, val_start + lookback + HORIZON)

    # reconstruct last lookback values (unscale)
    # we need scaler to invert: scaler.mean_ and scale_
    # Build a small helper to inverse-transform single-sample arrays
    def inv_scale_window(window_scaled):
        # window_scaled: (L, features)
        arr = np.array(window_scaled)
        mean = scaler.mean_
        scale = scaler.scale_
        return arr * scale + mean

    # reconstruct the ground truth window
    window_start = val_start - lookback
    raw_window = df.iloc[window_start: val_start + HORIZON]
    x_times = raw_window.index

    # plot true y
    y_true = raw_window['y'].values
    plt.plot(x_times, y_true, label='Ground truth')

    # predicted sequences start at val_start
    pred_times = df.index[val_start: val_start + HORIZON]
    # LSTM preds
    ypred_lstm = preds_lstm[sample_idx]
    ypred_trans = preds_trans[sample_idx]
    plt.plot(pred_times, ypred_lstm, marker='o', linestyle='--', label='LSTM+Attn')
    plt.plot(pred_times, ypred_trans, marker='x', linestyle='--', label='Transformer')
    plt.plot(pred_times, sarima_preds, marker='s', linestyle='--', label='SARIMA')

    plt.title(f'Fold {fold_id} — example forecast (horizon={HORIZON})')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f'fold{fold_id}_forecast.png'))
    plt.close()

    # attention heatmap for the example sample (if available)
    if attn_weights is not None:
        attn_sample = attn_weights[sample_idx]  # (seq_len,)
        plt.figure(figsize=(10,2))
        sns.heatmap(attn_sample.reshape(1, -1), cmap='viridis', cbar=True, xticklabels=False)
        plt.title(f'Fold {fold_id} — Attention weights (example sample)')
        plt.xlabel('Encoder time step')
        plt.savefig(os.path.join(OUTPUT_DIR, f'fold{fold_id}_attention.png'))
        plt.close()

# -----------------------------
# Report generation
# -----------------------------

def save_aggregate_report(all_results):
    # generate a small plain-text technical report summarizing metrics
    lines = []
    lines.append('Technical report — Time Series Forecasting with Attention')
    lines.append('\nSummary of folds:\n')
    for r in all_results:
        lines.append(f"Fold {r['fold']} — LSTM flat RMSE: {r['metrics_lstm']['flat']['rmse']:.4f}, Transformer flat RMSE: {r['metrics_transformer']['flat']['rmse']:.4f}, SARIMA flat RMSE: {r['metrics_sarima']['flat']['rmse']:.4f}")
    lines.append('\nNotes:')
    lines.append('- Models trained with PyTorch. LSTM model includes Bahdanau-style attention to provide interpretability (attention weights).')
    lines.append('- Rolling-origin (expanding window) evaluation used across multiple folds.')
    lines.append('- Metrics reported: RMSE, MAE, MAPE (per-horizon and flattened).')
    report_text = '\n'.join(lines)
    with open(os.path.join(OUTPUT_DIR, 'technical_report.txt'), 'w') as f:
        f.write(report_text)
    print('Saved technical report to', os.path.join(OUTPUT_DIR, 'technical_report.txt'))

if __name__ == '__main__':
    run_pipeline()

